<img src="images/stable_marriage.png" align=right width=30%></img>
# College Admissions and the Stability of Marriage
Author: Jin Yeom (jinyeom@utexas.edu)

## Contents
- [Stable marriage problem](#Stable-marriage-problem)
- [College admission problem](#College-admission-problem)
- [References](#References)

In [1]:
import random
from pprint import pprint
from copy import deepcopy
from typing import Mapping, Sequence, Tuple
from collections import defaultdict

In this notebook, we'll solve a little problem my girlfriend's sorority is having: matching new littles and bigs. Fortunately, the same problem has been solved long time ago by mathematicians named David Gale and Lloyd Shapley [1]. They called this problem the **stable marriage problem**, or the **college admissions problem** for more general cases.

## Stable marriage problem

*Given n men and n women, where each person has ranked all members of the opposite sex in order of preference, marry the men and women together such that there are no two people of opposite sex who would both rather have each other than their current partners. When there are no such pairs of people, the set of marriages is deemed stable.*

In [2]:
# Example from https://en.wikipedia.org/wiki/Stable_marriage_problem
# The preferences are modified to make the problem a little more challenging.
suiter_prefs = {"A": ["Y", "X", "Z"],
                "B": ["Y", "Z", "X"],
                "C": ["X", "Z", "Y"]}

reviewer_prefs = {"X": ["B", "A", "C"],
                  "Y": ["C", "B", "A"],
                  "Z": ["C", "A", "B"]}

In [3]:
def gale_shapley(suiter_prefs: Mapping[str, Sequence[str]], 
                 reviewer_prefs: Mapping[str, Sequence[str]]) -> Mapping[str, str]:
    suiters = list(suiter_prefs.keys())
    reviewers = list(reviewer_prefs.keys())
    # NOTE: this may seem wasteful, but it's either this or linear-time
    # search over suiters within each loop; we wouldn't want that, would we?
    suiter_matches = dict([(s, None) for s in suiters])
    reviewer_matches = dict([(r, None) for r in reviewers])
    while suiters:
        suiter = suiters.pop(0)
        reviewer = suiter_prefs[suiter][0]
        # if the current reviewer is not matched yet, match them
        if reviewer not in suiter_matches.values():
            suiter_matches[suiter] = reviewer
            reviewer_matches[reviewer] = suiter
        # otherwise, compare current matches of the two with each other
        else:
            suiter_ = reviewer_matches[reviewer]
            if reviewer_prefs[reviewer].index(suiter) < reviewer_prefs[reviewer].index(suiter_):
                suiter_matches[suiter_] = None
                suiter_matches[suiter] = reviewer
                reviewer_matches[reviewer] = suiter
                suiters.append(suiter_)
            else:
                suiter_prefs[suiter].remove(reviewer)
                suiters.append(suiter)
    return suiter_matches

# let's test it with the simple example from above
pprint(gale_shapley(suiter_prefs, reviewer_prefs))

{'A': 'X', 'B': 'Y', 'C': 'Z'}


Now, let's test the algorithm with a larger problem size.

In [4]:
suiters = ["Abigail", "Beth", "Chloe", "Daisy"]
reviewers = ["Emily", "Faith", "Gaby", "Haley"]
suiter_prefs = dict([(s, random.sample(reviewers, len(reviewers))) for s in suiters])
reviewer_prefs = dict([(r, random.sample(suiters, len(suiters))) for r in reviewers])

pprint(suiter_prefs)
pprint(reviewer_prefs)

{'Abigail': ['Gaby', 'Haley', 'Faith', 'Emily'],
 'Beth': ['Faith', 'Haley', 'Gaby', 'Emily'],
 'Chloe': ['Faith', 'Emily', 'Haley', 'Gaby'],
 'Daisy': ['Gaby', 'Faith', 'Haley', 'Emily']}
{'Emily': ['Beth', 'Chloe', 'Daisy', 'Abigail'],
 'Faith': ['Abigail', 'Daisy', 'Chloe', 'Beth'],
 'Gaby': ['Chloe', 'Daisy', 'Beth', 'Abigail'],
 'Haley': ['Daisy', 'Abigail', 'Beth', 'Chloe']}


In [5]:
pprint(gale_shapley(suiter_prefs, reviewer_prefs))

{'Abigail': 'Haley', 'Beth': 'Emily', 'Chloe': 'Faith', 'Daisy': 'Gaby'}


## College admissions problem

~~College admissions problem is a more general variation of stable marriage problem~~ (this statement is proven to be not necessarily true [2]), in which "suiters" and "reviewers" do not have to be of the same numbers, and each suiter (or reviewer) is not required to have all reviewers ranked in its preferences (and vice versa). Each reviwer also has a capacity of how many suiters it can be matched with at a time. This problem is also known as the hospital/resident problem.

In [6]:
def prefs(A: Sequence[str], 
          B: Sequence[str], 
          size: int) -> Tuple[Mapping[str, Sequence[str]]]:
    r"""Generate a tuple of random preferences of given size for each other"""
    a_prefs = dict([(a, random.sample(B, size)) for a in A])
    b_prefs = dict([(b, random.sample(A, size)) for b in B])
    return a_prefs, b_prefs

In [52]:
suiter_prefs, reviewer_prefs = prefs(suiters, reviewers, 3)
capacities = dict((s, random.choice([1, 2])) for s in suiters)
pprint(suiter_prefs)
pprint(reviewer_prefs)
pprint(capacities)

{'Abigail': ['Faith', 'Emily', 'Gaby'],
 'Beth': ['Haley', 'Emily', 'Faith'],
 'Chloe': ['Emily', 'Gaby', 'Haley'],
 'Daisy': ['Emily', 'Gaby', 'Faith']}
{'Emily': ['Daisy', 'Beth', 'Chloe'],
 'Faith': ['Chloe', 'Daisy', 'Beth'],
 'Gaby': ['Daisy', 'Abigail', 'Beth'],
 'Haley': ['Abigail', 'Chloe', 'Beth']}
{'Abigail': 2, 'Beth': 2, 'Chloe': 1, 'Daisy': 2}


In [49]:
def gale_shapley_cap(hospital_prefs: Mapping[str, Sequence[str]],
                     resident_prefs: Mapping[str, Sequence[str]],
                     capacities: Mapping[str, int]) -> Mapping[str, Sequence[str]]:
    r"""Gale-Shapley algorithm for solving the college admissions problem. 
    Note that this algorithm is different from Gale-Shapley algorithm for 
    solving the stable marriage problem.
    
    Args:
        hospital_prefs: a dict for preferences of each hospital
        resident_prefs: a dict for preferences of each resident
        capacities: a dict for capacities of each hospital
        
    Returns:
        a dict that maps each hospital with a list of residents
    """
    # preprocess preferences to pretend that both hospitals and residents each has a full
    # list of preferences of the other side
    hs = set(hospital_prefs.keys())
    rs = set(resident_prefs.keys())
    for h, rprefs in hospital_prefs.items():
        hospital_prefs[h].extend(list(rs - set(rprefs)))
    for r, hprefs in resident_prefs.items():
        resident_prefs[r].extend(list(hs - set(hprefs)))
    
    matches = defaultdict(list)
    unmatched_residents = list(resident_prefs.keys())
    while unmatched_residents:
        resident = unmatched_residents[0]
        hospital = resident_prefs[resident][0]
        matches[hospital].append(resident)
        
        if len(matches[hospital]) > capacities[hospital]:
            # if the hospital is matched with more residents than it can hold,
            # remove the least preferred resident from its matches
            least_pref = max(i for i, r in enumerate(hospital_prefs[hospital]) if r in matches[hospital])
            matches[hospital].remove(hospital_prefs[hospital][least_pref])
            
        if len(matches[hospital]) == capacities[hospital]:
            least_pref = max(i for i, r in enumerate(hospital_prefs[hospital]) if r in matches[hospital])
            for resident in hospital_prefs[hospital][least_pref+1:]:
                hospital_prefs[hospital].remove(resident)
                if hospital in resident_prefs[resident]:
                    resident_prefs[resident].remove(hospital)
                
        # update the unmatched residents
        unmatched_residents = [r for r in resident_prefs if resident_prefs[r] and 
                               not any([r in rs for rs in matches.values()])]
        
    return matches

In [53]:
pprint(gale_shapley_cap(suiter_prefs, reviewer_prefs, capacities))

defaultdict(<class 'list'>,
            {'Abigail': ['Haley'],
             'Chloe': ['Faith'],
             'Daisy': ['Emily', 'Gaby']})


## References

1. D. Gale and L. S. Shapley, "College Admissions and the Stability of Marriage", The American Mathematical Monthly, Vol. 69, No. 1 (Jan., 1962), pp. 9-15, Mathematical Association of America
2. Roth, A. E. "The College Admissions Problem Is Not Equivalent to the Marriage Problem." Journal of Economic Theory 36 (August 1985): 277–288.